In [ ]:
import os
import json
import csv
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir = '/content/drive/MyDrive/Json'
%cd '/content/drive/MyDrive/Json'

/content/drive/MyDrive/Json


In [ ]:
src = []
dest = []
stime = []
dur = []
wait = []
sendpoint = []
dendpoint = []
mark = []

In [ ]:
def recAdd(x):
  try:
      fro = x['childSpans'][0]['childSpans'][0]['source']['endpoint']['type']
      to = x['childSpans'][0]['childSpans'][0]['destination']['endpoint']['type']
      if fro == 'HTTP' and to == 'DATABASE':
        mark.append(1)
      else:
        mark.append(0)
  except:
      mark.append(0)
  for child in x['childSpans']:
    src.append(child["source"]["service"]["label"])
    dest.append(child["destination"]["service"]["label"])
    stime.append(child["calculatedSelfTime"])
    dur.append(child["duration"])
    sendpoint.append(child["source"]["endpoint"]["type"])
    dendpoint.append(child["source"]["endpoint"]["type"])
    recAdd(child)

In [ ]:
def waitAdd(x):
  total = 0
  for child in x['childSpans']:
    total += child["duration"]
  wait.append(total)
  for child in x['childSpans']:
    waitAdd(child)

In [ ]:
%pwd

'/content/drive/MyDrive/Json'

In [ ]:
idSet = set()
for fi in os.listdir(dir):
  f = open(fi)
  d = json.load(f)
  if d['id'] in idSet:
    continue
  idSet.add(d['id'])
  d = d['rootSpan']
  src.append(d["source"]["service"]["label"])
  dest.append(d["destination"]["service"]["label"])
  stime.append(d["calculatedSelfTime"])
  dur.append(d["duration"])
  sendpoint.append(d["source"]["endpoint"]["type"])
  dendpoint.append(d["source"]["endpoint"]["type"])
  recAdd(d)
  waitAdd(d)

In [ ]:
len(dur)

158

In [ ]:
new_src = []
new_dest = []
new_stime = []
new_dur = []
new_wait = []
new_sendpoint = []
new_dendpoint = []

flag = False
for i in range(len(src)):
  if flag:
    flag = False
    continue
  new_src.append(src[i])
  new_dest.append(dest[i])
  new_dur.append(dur[i])
  new_sendpoint.append(sendpoint[i])
  new_dendpoint.append(dendpoint[i])
  if mark[i] == 1:  
    new_stime.append(stime[i + 1])
    new_wait.append(wait[i + 1])
    flag = True
  else:
    new_stime.append(stime[i])
    new_wait.append(wait[i])

In [ ]:
len(new_dur)

123

In [ ]:
for i in range(len(new_dur)):
  print(str(new_dur[i]-new_stime[i]-new_wait[i]) + " " + str(new_src[i]) + " " + str(new_dest[i]) + " " + str(new_dur[i]) + " " + str(new_stime[i]) + " " + str(new_sendpoint[i]) + " " + str(new_dendpoint[i]))

0  nginx-web 3542 3 UNDEFINED UNDEFINED
0 nginx-web payment 3539 34 HTTP HTTP
10 payment rs-user 16 4 HTTP HTTP
0 rs-user rs-mongodb 2 2 HTTP HTTP
0 payment paypal.com 2236 2236 HTTP HTTP
0 payment www.paypal.com 85 85 HTTP HTTP
0 payment www.paypal.com 1125 1125 HTTP HTTP
0 payment rabbitmq 2 2 HTTP HTTP
3 payment rs-user 34 8 HTTP HTTP
0 rs-user rs-mongodb 2 2 HTTP HTTP
0 rs-user rs-mongodb 1 1 HTTP HTTP
0 rs-user rs-mongodb 20 20 HTTP HTTP
4 payment rs-cart 7 2 HTTP HTTP
0 rs-cart redis 1 1 HTTP HTTP
0  nginx-web 0 0 UNDEFINED UNDEFINED
0  nginx-web 546 2 UNDEFINED UNDEFINED
0 nginx-web rs-shipping 544 449 HTTP HTTP
78 rs-shipping rs-cart 95 16 HTTP HTTP
0 rs-cart redis 1 1 HTTP HTTP
0 rs-cart redis 0 0 HTTP HTTP
2  nginx-web 34 31 UNDEFINED UNDEFINED
0 rs-shipping rs-mysql-db 1 1 HTTP HTTP
1  rs-ratings 5 4 UNDEFINED UNDEFINED
0 rs-ratings rs-mysql-db 0 0 HTTP HTTP
0 rs-ratings rs-mysql-db 0 0 HTTP HTTP
2  nginx-web 64 8 UNDEFINED UNDEFINED
0 rs-shipping rs-mysql-db 54 54 HTTP HTTP

In [ ]:
topop = set()

In [ ]:
for i in range(len(new_dur)):
  if new_dur[i]-new_stime[i]-new_wait[i] < 0:
    topop.add(i)

In [ ]:
for i in range(len(new_dur)):
  if not(new_sendpoint[i] == 'HTTP' and new_dendpoint[i] == 'HTTP'):
    topop.add(i)
  if not new_src[i] or not new_dest[i]:
    topop.add(i)

In [ ]:
poplist = list(topop)
poplist.sort(reverse = True)

In [ ]:
for i in poplist:
  new_src.pop(i)
  new_dest.pop(i)
  new_stime.pop(i)
  new_dur.pop(i)
  new_wait.pop(i)
  new_sendpoint.pop(i)
  new_dendpoint.pop(i)

In [ ]:
bigList = [[] for x in range(len(new_src))]

In [ ]:
for i in range(len(new_src)):
  bigList[i].append(new_src[i])
  bigList[i].append(new_dest[i])
  bigList[i].append(new_dur[i])
  bigList[i].append(new_stime[i])
  bigList[i].append(new_wait[i])
  bigList[i].append(new_dur[i]-new_stime[i]-new_wait[i])

In [ ]:
bigList

[['nginx-web', 'payment', 3539, 34, 3505, 0],
 ['payment', 'rs-user', 16, 4, 2, 10],
 ['rs-user', 'rs-mongodb', 2, 2, 0, 0],
 ['payment', 'paypal.com', 2236, 2236, 0, 0],
 ['payment', 'www.paypal.com', 85, 85, 0, 0],
 ['payment', 'www.paypal.com', 1125, 1125, 0, 0],
 ['payment', 'rabbitmq', 2, 2, 0, 0],
 ['payment', 'rs-user', 34, 8, 23, 3],
 ['rs-user', 'rs-mongodb', 2, 2, 0, 0],
 ['rs-user', 'rs-mongodb', 1, 1, 0, 0],
 ['rs-user', 'rs-mongodb', 20, 20, 0, 0],
 ['payment', 'rs-cart', 7, 2, 1, 4],
 ['rs-cart', 'redis', 1, 1, 0, 0],
 ['nginx-web', 'rs-shipping', 544, 449, 95, 0],
 ['rs-shipping', 'rs-cart', 95, 16, 1, 78],
 ['rs-cart', 'redis', 1, 1, 0, 0],
 ['rs-cart', 'redis', 0, 0, 0, 0],
 ['rs-shipping', 'rs-mysql-db', 1, 1, 0, 0],
 ['rs-ratings', 'rs-mysql-db', 0, 0, 0, 0],
 ['rs-ratings', 'rs-mysql-db', 0, 0, 0, 0],
 ['rs-shipping', 'rs-mysql-db', 54, 54, 0, 0],
 ['rs-shipping', 'rs-mysql-db', 62, 62, 0, 0],
 ['rs-ratings', 'rs-mysql-db', 0, 0, 0, 0],
 ['rs-ratings', 'rs-mysql-db'

In [ ]:
info = dict()
for i in bigList:
  temp = (i[0], i[1])
  if temp in info:
    info[temp]['stime'] += i[3]
    info[temp]['network'] += i[5]
    info[temp]['calls'] += 1
  else:
    info[temp] = dict()
    info[temp]['stime'] = i[3]
    info[temp]['network'] = i[5]
    info[temp]['calls'] = 1

for edge in info:
  info[edge]['stime'] /= info[edge]['calls']
  info[edge]['network'] /= info[edge]['calls']

info

{('nginx-web', 'payment'): {'stime': 34.0, 'network': 0.0, 'calls': 1},
 ('payment', 'rs-user'): {'stime': 6.0, 'network': 6.5, 'calls': 2},
 ('rs-user', 'rs-mongodb'): {'stime': 5.333333333333333,
  'network': 0.0,
  'calls': 6},
 ('payment', 'paypal.com'): {'stime': 2236.0, 'network': 0.0, 'calls': 1},
 ('payment', 'www.paypal.com'): {'stime': 605.0, 'network': 0.0, 'calls': 2},
 ('payment', 'rabbitmq'): {'stime': 2.0, 'network': 0.0, 'calls': 1},
 ('payment', 'rs-cart'): {'stime': 2.0, 'network': 4.0, 'calls': 1},
 ('rs-cart', 'redis'): {'stime': 0.7777777777777778,
  'network': 0.0,
  'calls': 9},
 ('nginx-web', 'rs-shipping'): {'stime': 449.0, 'network': 0.0, 'calls': 1},
 ('rs-shipping', 'rs-cart'): {'stime': 16.0, 'network': 78.0, 'calls': 1},
 ('rs-shipping', 'rs-mysql-db'): {'stime': 30.0, 'network': 0.0, 'calls': 4},
 ('rs-ratings', 'rs-mysql-db'): {'stime': 0.2222222222222222,
  'network': 0.0,
  'calls': 18},
 ('nginx-web', 'rs-cart'): {'stime': 7.5, 'network': 0.0, 'calls'

In [ ]:
finalList = [[] for i in range(len(info))]
i = 0
for edge in info:
  finalList[i].append(edge[0])
  finalList[i].append(edge[1])
  finalList[i].append(info[edge]['stime'])
  finalList[i].append(info[edge]['network'])
  finalList[i].append(info[edge]['calls'])
  i += 1
finalList

[['nginx-web', 'payment', 34.0, 0.0, 1],
 ['payment', 'rs-user', 6.0, 6.5, 2],
 ['rs-user', 'rs-mongodb', 5.333333333333333, 0.0, 6],
 ['payment', 'paypal.com', 2236.0, 0.0, 1],
 ['payment', 'www.paypal.com', 605.0, 0.0, 2],
 ['payment', 'rabbitmq', 2.0, 0.0, 1],
 ['payment', 'rs-cart', 2.0, 4.0, 1],
 ['rs-cart', 'redis', 0.7777777777777778, 0.0, 9],
 ['nginx-web', 'rs-shipping', 449.0, 0.0, 1],
 ['rs-shipping', 'rs-cart', 16.0, 78.0, 1],
 ['rs-shipping', 'rs-mysql-db', 30.0, 0.0, 4],
 ['rs-ratings', 'rs-mysql-db', 0.2222222222222222, 0.0, 18],
 ['nginx-web', 'rs-cart', 7.5, 0.0, 2],
 ['rs-cart', 'rs-catalogue', 4.5, 4.0, 2],
 ['rs-catalogue', 'rs-mongodb', 2.5, 0.0, 6],
 ['nginx-web', 'rs-ratings', 7.0, 1.0, 1],
 ['rs-ratings', 'ratings', 0.2, 0.0, 10],
 ['catalogue', 'rs-mongodb', 1.0, 0.0, 1],
 ['user', 'redis', 2.0, 0.0, 1]]

In [ ]:
columns = ['source', 'destination', 'self', 'network', 'calls']

In [ ]:
with open('Edge.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(columns)
    write.writerows(finalList)